![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Experiment-Notebook.png)
# Experiment Notebook -AutoAI Notebook v1.18.4


This notebook contains the steps and code to demonstrate support of AutoAI experiments in the Watson Machine Learning service. It introduces Python API commands for data retrieval, training experiments, persisting pipelines, testing pipelines, refining pipelines, and scoring the resulting model.

**Note:** Notebook code generated using AutoAI will execute successfully. If code is modified or reordered, there is no guarantee it will successfully execute. For details, see: <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Saving an Auto AI experiment as a notebook</a>


Some familiarity with Python is helpful. This notebook uses Python 3.10 and the `ibm_watson_machine_learning` package.


## Notebook goals

The learning goals of this notebook are:
-  Defining an AutoAI experiment
-  Training AutoAI models 
-  Comparing trained models
-  Deploying the model as a web service
-  Scoring the model to generate predictions



## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Experiment configuration](#configuration)**<br>
&nbsp;&nbsp;[Experiment metadata](#metadata)<br>
**[Working with completed AutoAI experiment](#work)**<br>
&nbsp;&nbsp;[Get fitted AutoAI optimizer](#get)<br>
&nbsp;&nbsp;[Pipelines comparison](#comparison)<br>
&nbsp;&nbsp;[Get pipeline as a scikit-learn pipeline model](#get_pipeline)<br>
&nbsp;&nbsp;[Inspect pipeline](#inspect_pipeline)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Visualize pipeline model](#visualize)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Preview pipeline model as a Python code](#preview)<br>
**[Deploy and Score](#scoring)**<br>
&nbsp;&nbsp;[Working with spaces](#working_spaces)<br>
**[Running AutoAI experiment with Python API](#run)**<br>
**[Clean up](#cleanup)**<br>
**[Next steps](#next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watson-machine-learning,
 - autoai-ts-libs


In [ ]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U 'autoai-ts-libs>=1.2.0,<4.0' | tail -n 1
!pip install -U scikit-learn==1.1.1 | tail -n 1
!pip install -U nose | tail -n 1
!pip install -U 'jupyter>=1' | tail -n 1

<a id="configuration"></a>
# Experiment configuration

<a id="metadata"></a>
## Experiment metadata
This cell defines the metadata for the experiment, including: training_data_references, training_result_reference, experiment_metadata.

In [ ]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='56952cea-e9b4-471b-b781-fd6772f653cf'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/d38cd7f1-addf-4a64-a90f-50ee5f0e0c6b/wml_data/18768e39-dda4-46f0-ade6-122a1cd5552d/data/autoai-ts',
        model_location='auto_ml/d38cd7f1-addf-4a64-a90f-50ee5f0e0c6b/wml_data/18768e39-dda4-46f0-ade6-122a1cd5552d/data/autoai-ts/model.zip',
        training_status='auto_ml/d38cd7f1-addf-4a64-a90f-50ee5f0e0c6b/wml_data/18768e39-dda4-46f0-ade6-122a1cd5552d/training-status.json'
    )
)

In [ ]:
experiment_metadata = dict(
    prediction_type='timeseries',
    prediction_columns=['MT_002', 'MT_004', 'MT_007'],
    csv_separator=',',
    holdout_size=20,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    timestamp_column_name='timestamp',
    backtest_num=4,
    pipeline_type='all',
    customized_pipelines=[],
    lookback_window=-1,
    forecast_window=1,
    max_num_daub_ensembles=3,
    feature_columns=['MT_002', 'MT_004', 'MT_007'],
    future_exogenous_available=True,
    gap_len=0,
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='e8f7bcf9-37a2-4880-b912-06970bf03a80',
    numerical_imputation_strategy=['FlattenIterative', 'Linear', 'Cubic', 'Previous']
)

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [ ]:
api_key = 'PUT_YOUR_APIKEY_HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

<a id="work"></a>


# Working with the completed AutoAI experiment

This cell imports the pipelines generated for the experiment. The best pipeline will be saved as a model.

<a id="get"></a>


## Get fitted AutoAI optimizer

In [ ]:
from ibm_watson_machine_learning.experiment import AutoAI

pipeline_optimizer = AutoAI(wml_credentials, project_id=experiment_metadata['project_id']).runs.get_optimizer(metadata=experiment_metadata)

Use `get_params()` to retrieve configuration parameters.

In [ ]:
pipeline_optimizer.get_params()

<a id="comparison"></a>
## Pipelines comparison

Use the `summary()` method to list trained pipelines and evaluation metrics information in
the form of a Pandas DataFrame. You can use the DataFrame to compare all discovered pipelines and select the one you like for further testing.

In [ ]:
summary = pipeline_optimizer.summary()
best_pipeline_name = list(summary.index)[0]
summary

<a id="get_pipeline"></a>
### Get pipeline as a scikit-learn pipeline model

After you compare the pipelines, download and save a scikit-learn pipeline model object from the
AutoAI training job.

**Tip:** To get a specific pipeline, pass the pipeline name in:
```
pipeline_optimizer.get_pipeline(pipeline_name=pipeline_name)
```

In [ ]:
pipeline_model = pipeline_optimizer.get_pipeline()

**Tip:** If you want to check all the details of the model evaluation metrics, use:
```
pipeline_optimizer.get_pipeline_details()
```

<a id="inspect_pipeline"></a>
## Inspect pipeline

<a id="visualize"></a>
### Visualize pipeline model

Preview pipeline model stages as a graph. Each node's name links to a detailed description of the stage.


In [ ]:
pipeline_model.visualize()

<a id="preview"></a>
### Preview pipeline model as a Python code
In the next cell, you can preview the saved pipeline model as a Python code.  
You can review the exact steps used to create the model.

**Note:** If you want to get sklearn representation, add the following parameter to the `pretty_print` call: `astype='sklearn'`.

In [ ]:
pipeline_model.pretty_print(combinators=False, ipython_display=True)

### Calling the `predict` method
If you want to get a prediction by using the pipeline model object, call `pipeline_model.predict()`.

**Note:** If you want to work with a pure sklearn model:
 - add the following parameter to the `get_pipeline` call: `astype='sklearn'`,
 - or `scikit_learn_pipeline = pipeline_model.export_to_sklearn_pipeline()`

<a id="scoring"></a>
## Deploy and Score

In this section you will learn how to deploy and score the model as a web service.

<a id="working_spaces"></a>
### Working with spaces

In this section you will specify a deployment space to organize assets and then deploy and score the model. If you do not have an existing space, you can use the [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create a new space. Follow these steps:

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press **Create**
- Copy `space_id` and paste it below

**Tip**: You can also use the API to prepare the space for your work. Learn more [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign or update space ID below.

### Deployment creation

In [ ]:
target_space_id = "PUT_YOUR_TARGET_SPACE_ID_HERE"

from ibm_watson_machine_learning.deployment import WebService

service = WebService(
    source_wml_credentials=wml_credentials,
    target_wml_credentials=wml_credentials,
    source_project_id=experiment_metadata['project_id'],
    target_space_id=target_space_id
)
service.create(
    model=best_pipeline_name,
    experiment_run_id=pipeline_optimizer.get_params()['run_id'],
    deployment_name='Best_pipeline_webservice'
)

Use the `print` method for the deployment object to show basic information about the service: 

In [ ]:
print(service)

To show all available information about the deployment, use the `.get_params()` method.

In [ ]:
service.get_params()

### Scoring of webservice
You can make a scoring request by calling `score()` on the deployed pipeline.

If you want to work with the web service in an external Python application, follow these steps to retrieve the service object:

 - Initialize the service by `service = WebService(target_wml_credentials=wml_credentials,target_space_id=experiment_metadata['space_id'])`
 - Get deployment_id: `service.list()`
 - Get webservice object: `service.get('deployment_id')`

After that you can call `service.score(score_records_df)` method. The `score()` method accepts `pandas.DataFrame` objects. 

<a id="cleanup"></a>
### Deleting deployment
You can delete the existing deployment by calling the `service.delete()` command.
To list the existing web services, use the `service.list()` method.

<a id="run"></a>

## Running the AutoAI experiment with Python API

If you want to run the AutoAI experiment using the Python API, follow these steps. The experiment settings were generated basing on parameters set in the AutoAI UI.


```
from ibm_watson_machine_learning.experiment import AutoAI

experiment = AutoAI(wml_credentials, project_id=experiment_metadata['project_id'])
```

```
OPTIMIZER_NAME = 'custom_name'
```

```
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='56952cea-e9b4-471b-b781-fd6772f653cf'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/d38cd7f1-addf-4a64-a90f-50ee5f0e0c6b/wml_data/18768e39-dda4-46f0-ade6-122a1cd5552d/data/autoai-ts',
        model_location='auto_ml/d38cd7f1-addf-4a64-a90f-50ee5f0e0c6b/wml_data/18768e39-dda4-46f0-ade6-122a1cd5552d/data/autoai-ts/model.zip',
        training_status='auto_ml/d38cd7f1-addf-4a64-a90f-50ee5f0e0c6b/wml_data/18768e39-dda4-46f0-ade6-122a1cd5552d/training-status.json'
    )
)
```

The new pipeline optimizer will be created and training will be triggered.

```
pipeline_optimizer = experiment.optimizer(
    name=OPTIMIZER_NAME,
    prediction_type=experiment_metadata['prediction_type'],
    prediction_columns=experiment_metadata['prediction_columns'],
    holdout_size=experiment_metadata['holdout_size'],
    max_number_of_estimators=experiment_metadata['max_num_daub_ensembles'],
    csv_separator=experiment_metadata['csv_separator'],
    lookback_window=experiment_metadata['lookback_window'],
    forecast_window=experiment_metadata['forecast_window'],
    timestamp_column_name=experiment_metadata['timestamp_column_name'],
    backtest_num=experiment_metadata['backtest_num'],
    feature_columns=experiment_metadata['feature_columns'],
    supporting_features_at_forecast=experiment_metadata['future_exogenous_available'],
    pipeline_types=['RandomForestRegressor', 'ExogenousRandomForestRegressor', 'SVM', 'ExogenousSVM', 'LocalizedFlattenEnsembler', 'DifferenceFlattenEnsembler', 'FlattenEnsembler', 'ExogenousLocalizedFlattenEnsembler', 'ExogenousDifferenceFlattenEnsembler', 'ExogenousFlattenEnsembler', 'MT2RForecaster', 'ExogenousMT2RForecaster', 'HoltWinterAdditive', 'HoltWinterMultiplicative', 'Bats', 'ARIMA', 'ARIMAX', 'ARIMAX_RSAR', 'ARIMAX_PALR', 'ARIMAX_RAR', 'ARIMAX_DMLR'],
    backtest_gap_length=experiment_metadata['gap_len'],
    numerical_imputation_strategy=experiment_metadata['numerical_imputation_strategy'],
)
```

```
pipeline_optimizer.fit(
    training_data_references=training_data_references,
    training_results_reference=training_result_reference,
)
```


<a id="next_steps"></a>
# Next steps
You successfully completed this notebook!
You learned how to use ibm-watson-machine-learning to run and explore AutoAI experiments.
Check out the official [AutoAI site](https://www.ibm.com/cloud/watson-studio/autoai) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>  

___